## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server.

In [ ]:
import mlflow
import os
#mlflow server -h 0.0.0.0 -p 5000 --backend-store-uri postgresql://DB_USER:DB_PASSWORD@DB_ENDPOINT:5432/DB_INITAL_NAME --default-artifact-root s3://S3_BUCKET_NAME
os.environ["AWS_PROFILE"] = "BartoszGrabarz01"

TRACKING_SERVER_HOST = "ec2-13-53-110-69.eu-north-1.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

exp_name = "my-exp-fixed"
artifact_loc = "s3://my-server-artifacts-bucket"

exp_id = client.create_experiment(exp_name, artifact_location=artifact_loc)
mlflow.set_experiment(exp_name)

print("Created experiment:", exp_name, "id:", exp_id)


Created experiment: my-exp-fixed id: 2


In [8]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-13-53-110-69.eu-north-1.compute.amazonaws.com:5000'


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://my-server-artifacts-bucket', creation_time=1768488663327, experiment_id='2', last_update_time=1768488663327, lifecycle_stage='active', name='my-exp-fixed', tags={}>,
 <Experiment: artifact_location='s3://my-server-arrtifacts-bucket/0', creation_time=1768479471365, experiment_id='0', last_update_time=1768479471365, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-exp-fixed")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2026/01/15 15:51:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


default artifacts URI: 's3://my-server-artifacts-bucket/8e3ff4f1773741b5bd0ee545816dc284/artifacts'
🏃 View run gregarious-ox-353 at: http://ec2-13-53-110-69.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/8e3ff4f1773741b5bd0ee545816dc284
🧪 View experiment at: http://ec2-13-53-110-69.eu-north-1.compute.amazonaws.com:5000/#/experiments/2


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://my-server-artifacts-bucket', creation_time=1768488663327, experiment_id='2', last_update_time=1768488663327, lifecycle_stage='active', name='my-exp-fixed', tags={}>,
 <Experiment: artifact_location='s3://my-server-arrtifacts-bucket/0', creation_time=1768479471365, experiment_id='0', last_update_time=1768479471365, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [13]:
client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [14]:
client.search_registered_models()

[]

In [15]:
run_id = client.search_runs(experiment_ids=['2'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2026/01/15 15:56:56 WARNING mlflow.tracking._model_registry.fluent: Run with id 8e3ff4f1773741b5bd0ee545816dc284 has no artifacts at artifact path 'models', registering model based on models:/m-895464a7b1e74c9dbbab7e89cc9025c0 instead
2026/01/15 15:56:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1768489016820, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1768489016820, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='8e3ff4f1773741b5bd0ee545816dc284', run_link='', source='models:/m-895464a7b1e74c9dbbab7e89cc9025c0', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [16]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1768489015470, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='', last_updated_timestamp=1768489016820, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1768489016820, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1768489016820, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='8e3ff4f1773741b5bd0ee545816dc284', run_link='', source='models:/m-895464a7b1e74c9dbbab7e89cc9025c0', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]